In [81]:
# 필요한 library 불러오기

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [82]:
train = pd.read_csv('train.csv')

In [83]:
test = pd.read_csv('test.csv')

In [84]:
sample_submission = pd.read_csv('sample_submission.csv')

In [85]:
age_gender_info = pd.read_csv('age_gender_info.csv')

In [86]:
train.shape

(2952, 15)

> 데이터의 수 자체가 적음, 전처리가 중요

In [87]:
train.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

> 총 4개의 column에서 결측치 발생

In [88]:
test.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

> 총 4개의 column에서 결측치발생

> train에서는 결측치가 발생하지않은 column : `자격유형`

In [89]:
train_nunique = train.nunique()
train_nunique

단지코드                            423
총세대수                            351
임대건물구분                            2
지역                               16
공급유형                             10
전용면적                            679
전용면적별세대수                        403
공가수                              48
자격유형                             15
임대보증금                           957
임대료                             995
도보 10분거리 내 지하철역 수(환승노선 수 반영)      4
도보 10분거리 내 버스정류장 수               17
단지내주차면수                         355
등록차량수                           354
dtype: int64

In [90]:
#데이터 타입과 unique한 값의 갯수 보기
train_type_nunique = pd.DataFrame({
    'Data Type': train.dtypes,
    'Number of Unique Values': train.nunique()
})

train_type_nunique

,Data Type,Number of Unique Values
단지코드,object,423
총세대수,int64,351
임대건물구분,object,2
지역,object,16
공급유형,object,10
전용면적,float64,679
전용면적별세대수,int64,403
공가수,float64,48
자격유형,object,15
임대보증금,object,957


> object이면서 nunique가 많은 값 : `공급유형`, `자격유형`, `지역`

In [91]:
# 지역 data 합치기

train.loc[train.지역.isin(['경기도','서울특별시']), '지역'] = '수도권(서울/경기)'
train.loc[train.지역.isin(['경상남도','울산광역시','부산광역시','대구광역시','경상북도']), '지역'] = '경상도'
train.loc[train.지역.isin(['전라남도','광주광역시','전라북도']), '지역'] = '전라도'
train.loc[train.지역.isin(['충청남도','충청북도','대전광역시','세종특별자치시']), '지역'] = '충청도'

test.loc[test.지역.isin(['경기도','서울특별시']), '지역'] = '수도권(서울/경기)'
test.loc[test.지역.isin(['경상남도','울산광역시','부산광역시','대구광역시','경상북도']), '지역'] = '경상도'
test.loc[test.지역.isin(['전라남도','광주광역시','전라북도']), '지역'] = '전라도'
test.loc[test.지역.isin(['충청남도','충청북도','대전광역시','세종특별자치시']), '지역'] = '충청도'

In [92]:
train['공급유형'].unique()

array(['국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'], dtype=object)

In [93]:
# train, test column rename : column명이 긴 것을 간결화
train.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)': '지하철수',
                      '도보 10분거리 내 버스정류장 수': '버스수'}, inplace=True)
test.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)': '지하철수',
                     '도보 10분거리 내 버스정류장 수': '버스수'}, inplace=True)


# 결측치 통일 : '임대료','임대보증금' 컬럼에서 '-' 값을 NaN으로 변경

for col in ['임대료', '임대보증금']:
    train[col] = train[col].replace('-', np.nan).astype(float)
    test[col] = test[col].replace('-', np.nan).astype(float)

# 해당 컬럼을 float64 타입으로 변환
train['임대료'] = train['임대료'].astype(float)
train['임대보증금'] = train['임대보증금'].astype(float)

# 해당 컬럼을 float64 타입으로 변환
test['임대료'] = test['임대료'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

# train - 버스수는 10으로, 지하철수는 0으로 결측치를 채움
train[train['지하철수'].isna()]['지역'].unique() # 충남, 대전, 경남으로 이 중에서 대전에만 지하철 운행 - 충남, 경남 0으로 채우기로 결정

# 대전광역시 단지코드 확인
nan_subway = train[(train['지역'] == '대전광역시') & (train['지하철수'].isna())]
nan_subway_codes = nan_subway['단지코드'].unique()
print(nan_subway_codes)
# 전국 LH아파트 단지정보와 비교하여 도보 10분 내 지하철 역 없는 것 확인, 0으로 채움

#대전광역시 동구 우암로 133  (주공삼성타운아파트)
# 대전광역시 중구 목중로39번길 35-14(목동) 대전목동 행복주택
# 버스도 전국 LH아파트 단지정보와 비교하여 [경상남도 양산시 물금읍 새실로 54(양산신도시엘에이치아파트5단지)] 10개 정도 도보로 가능함을 확인

train['버스수'].fillna(10, inplace=True)
train['지하철수'].fillna(0, inplace=True)

# test - 지하철수 0으로 결측치를 채움
test[test['지하철수'].isna()]['지역'].unique() # 충남, 대전으로 이 중에서 대전에만 지하철 운행 - 충남 0으로 채우기로 결정

# 대전광역시 단지코드 확인
nan_subway = test[(test['지역'] == '대전광역시') & (train['지하철수'].isna())]
nan_subway_codes = nan_subway['단지코드'].unique()
print(nan_subway_codes)
# 전국 LH아파트 단지정보와 비교하여 도보 10분 내 지하철 역 없는 것 확인, 0으로 채움 [대전광역시 동구 우암로 133  (주공삼성타운아파트), 대전광역시 중구 목중로39번길 35-14(목동) 대전목동 행복주택]]

test['지하철수'].fillna(0, inplace=True)

#자격유형 결측치 확인
test[test['자격유형'].isna()]

#C2411 자격유형 → 'A'
test[test['단지코드']=='C2411'] #C2411 자격유형이 모두 A로, 단순 결측치인 것으로 판단
test[test['단지코드']=='C2411']['자격유형'].fillna('A')

#C2253 자격유형 → 'C'
test[test['단지코드']=='C2253'] #자격유형 C, D를 구분하는 컬럼이 '임대건물구분'이라고 판단, 결측치는 임대건물구분이 아파트이기 때문에 자격유형 C로 채워넣음
test[test['단지코드']=='C2253']['자격유형'].fillna('C')

train.head()

[]
[]


<ipython-input-93-ccd230def84a>:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  nan_subway = test[(test['지역'] == '대전광역시') & (train['지하철수'].isna())]


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철수,버스수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상도,국민임대,39.72,134,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상도,국민임대,39.72,15,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상도,국민임대,51.93,385,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상도,국민임대,51.93,15,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상도,국민임대,51.93,41,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0


In [94]:
train[train['지하철수'].isna()]['지역'].unique()

array([], dtype=object)

In [95]:
# 임대료 혹은 임대보증금이 NaN값인 행 필터링
missing_data = train[train['임대료'].isna() | train['임대보증금'].isna()]

# 모든 열에 대해 통계요약값과 누락 데이터 몇 행정도 표시
summary = missing_data.describe(include='all')
summary


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철수,버스수,단지내주차면수,등록차량수
count,590,590.000000,590,590,590,590.000000,590.000000,590.000000,590,9.000000e+00,0.0,590.000000,590.000000,590.000000,590.000000
unique,39,NaN,2,5,5,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
top,C1439,NaN,상가,경상도,임대상가,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN
freq,45,NaN,562,325,562,NaN,NaN,NaN,569,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1236.922034,NaN,NaN,NaN,45.542356,7.984746,7.886441,NaN,1.842430e+08,NaN,0.288136,3.679661,313.910169,213.735593
std,NaN,652.934635,NaN,NaN,NaN,66.314353,45.403471,10.514231,NaN,3.962353e+07,NaN,0.453279,2.308343,293.986239,308.756185
min,NaN,370.000000,NaN,NaN,NaN,12.620000,1.000000,0.000000,NaN,8.744400e+07,NaN,0.000000,1.000000,65.000000,31.000000
25%,NaN,657.000000,NaN,NaN,NaN,26.250000,1.000000,1.000000,NaN,1.748880e+08,NaN,0.000000,2.000000,153.000000,94.000000
50%,NaN,1005.000000,NaN,NaN,NaN,31.840000,1.000000,2.000000,NaN,1.945620e+08,NaN,0.000000,3.000000,217.000000,127.000000
75%,NaN,1755.000000,NaN,NaN,NaN,37.950000,1.000000,9.000000,NaN,2.138630e+08,NaN,1.000000,4.000000,351.000000,189.000000


In [96]:
print("임대건물구분이 상가인 것의 임대료",train[train['임대건물구분'] == '상가']['임대료'].unique())
print("임대건물구분이 상가인 것의 임대보증금",train[train['임대건물구분'] == '상가']['임대보증금'].unique())
print("공급유형이 임대상가인 것의 임대료",train[train['공급유형']=='임대상가']['임대료'].unique())
print("공급유형이 임대상가인 것의 임대보증금",train[train['공급유형']=='임대상가']['임대보증금'].unique())
print("자격유형이 D인 것의 임대료",train[train['자격유형'] == 'D']['임대료'].unique())
print("자격유형이 D인 것의 임대보증금",train[train['자격유형'] == 'D']['임대보증금'].unique())

임대건물구분이 상가인 것의 임대료 [nan]
임대건물구분이 상가인 것의 임대보증금 [nan]
공급유형이 임대상가인 것의 임대료 [nan]
공급유형이 임대상가인 것의 임대보증금 [nan]
자격유형이 D인 것의 임대료 [nan]
자격유형이 D인 것의 임대보증금 [nan]


> 해당하는 조건을 해봤으나 모두 nan값이라 유의미한 결과를 도출할 수 없음

> 상가가 아닌 것의 데이터를 살펴보았습니다.

In [97]:
train[(train['임대료'].isna()) & (train['임대건물구분'] != '상가')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철수,버스수,단지내주차면수,등록차량수
2043,C1397,370,아파트,수도권(서울/경기),장기전세,23.32,51,9.0,A,87444000.0,NaN,0.0,3.0,1590.0,1595.0
2044,C1397,370,아파트,수도권(서울/경기),장기전세,46.79,4,9.0,A,174888000.0,NaN,0.0,3.0,1590.0,1595.0
2045,C1397,370,아파트,수도권(서울/경기),장기전세,46.91,69,9.0,A,174888000.0,NaN,0.0,3.0,1590.0,1595.0
2046,C1397,370,아파트,수도권(서울/경기),장기전세,51.96,24,9.0,A,194562000.0,NaN,0.0,3.0,1590.0,1595.0
2047,C1397,370,아파트,수도권(서울/경기),장기전세,51.99,80,9.0,A,194562000.0,NaN,0.0,3.0,1590.0,1595.0
2048,C1397,370,아파트,수도권(서울/경기),장기전세,59.93,142,9.0,A,216423000.0,NaN,0.0,3.0,1590.0,1595.0
2100,C1039,790,아파트,수도권(서울/경기),장기전세,51.32,126,13.0,A,187694000.0,NaN,0.0,3.0,673.0,645.0
2101,C1039,790,아파트,수도권(서울/경기),장기전세,59.88,49,13.0,A,213863000.0,NaN,0.0,3.0,673.0,645.0
2102,C1039,790,아파트,수도권(서울/경기),장기전세,59.94,75,13.0,A,213863000.0,NaN,0.0,3.0,673.0,645.0
2331,C1350,1401,아파트,충청도,공공분양,74.94,317,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0


In [98]:
test[(test['임대료'].isna()) & (test['임대건물구분'] != '상가')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철수,버스수,단지내주차면수
76,C1006,1505,아파트,충청도,영구임대,26.37,10,27.0,D,NaN,NaN,2.0,5.0,428.0
77,C1006,1505,아파트,충청도,영구임대,26.37,10,27.0,D,NaN,NaN,2.0,5.0,428.0
79,C1006,1505,아파트,충청도,영구임대,52.74,6,27.0,D,NaN,NaN,2.0,5.0,428.0
1005,C2152,120,아파트,강원도,영구임대,24.83,66,9.0,C,NaN,NaN,0.0,1.0,40.0
1006,C2152,120,아파트,강원도,영구임대,33.84,54,9.0,C,NaN,NaN,0.0,1.0,40.0
1014,C1267,675,아파트,경상도,행복주택,16.94,50,38.0,L,NaN,NaN,0.0,1.0,467.0
1015,C1267,675,아파트,경상도,행복주택,26.85,66,38.0,L,NaN,NaN,0.0,1.0,467.0
1016,C1267,675,아파트,경상도,행복주택,26.85,8,38.0,L,NaN,NaN,0.0,1.0,467.0
1017,C1267,675,아파트,경상도,행복주택,36.77,126,38.0,L,NaN,NaN,0.0,1.0,467.0


> 다른 행들은 임대보증금과 임대료가 모두 결측치이지만 장기전세는 임대료만 결측치인 것을 확인할 수 있습니다.

> 장기전세주택이란 월 임대료를 지불하지 않고 전세계약 방식으로 공급되는 공공임대주택을 말합니다. [출처](https://www.mylawstory.com/5606/)

>그러므로 장기전세의 임대료는 0으로 바꿔주는 것이 맞습니다


In [99]:
#### 장기전세는 월 임대료를 지불하지않으므로 임대료 = 0 적용
train.loc[train['공급유형'] == '장기전세', '임대료'] = 0
test.loc[test['공급유형'] == '장기전세', '임대료'] = 0

> 단지내 주차면수과 임대료, 임대보증금이 높은 상관관계를 가지므로

> 공급유형이 상가가 아닌것들 중에 임대료와 임대보증금이 결측치인경우 지역별로 나눈 후 단지내 주차면수에 따른 회귀모델을 통해 임대료와 임대보증금의 결측치를 채워줍니다.

In [100]:
#train data 상가아닌 것 임대료, 임대보증금 회귀예측

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 공급유형이 상가가 아닌 행만 남도록 필터링
train_non_store = train[train['공급유형'] != '상가']

# train_non_store 데이터의 지역명 확인
regions = train_non_store['지역'].unique()

# 예측값 저장을 위해 빈 리스트 생성
predicted_rent = [] #임대료 예측값 리스트
predicted_deposit = [] #임대보증금 예측값 리스트

# 지역을 기준으로 반복
for region in regions:
    region_data = train_non_store[train_non_store['지역'] == region].copy()

    # 임대료에 대한 회귀 모델 학습
    # 임대료 열에 결측값이 없는 데이터를 선택하고, 단지내주차면수을 X, 임대료를 y로 사용
    X = region_data[~region_data['임대료'].isna()][['단지내주차면수']]
    y = region_data[~region_data['임대료'].isna()]['임대료']
    if len(X) > 0:
        model_rent = LinearRegression()
        model_rent.fit(X, y)
        # 결측치 예측
        # 임대료 열에 결측치 있는 데이터 선택하여 전용면적을 X로 사용하여 임대료 예측
        X_missing_rent = region_data[region_data['임대료'].isna()][['단지내주차면수']]
        if len(X_missing_rent) > 0:
            pred_rent = model_rent.predict(X_missing_rent)
            predicted_rent.extend(pred_rent)

    # 임대보증금에 대한 회귀 모델 학습
    # 임대보증금 열에 결측값이 없는 데이터를 선택하고, 전용면적을 X, 임대보증금를 y로 사용
    X = region_data[~region_data['임대보증금'].isna()][['단지내주차면수']]
    y = region_data[~region_data['임대보증금'].isna()]['임대보증금']
    if len(X) > 0:
        model_deposit = LinearRegression()
        model_deposit.fit(X, y)
        # 결측치 예측
        X_missing_deposit = region_data[region_data['임대보증금'].isna()][['단지내주차면수']]
        # 데이터가 존재하면 모델을 사용해 임대료를 예측하고 리스트에 추가하도록 함
        if len(X_missing_deposit) > 0:
            pred_deposit = model_deposit.predict(X_missing_deposit)
            predicted_deposit.extend(pred_deposit)

# train_non_store의 임대료와 임대보증금이 결측치인 것들을 채워넣음
missing_rent_indices = train_non_store[train_non_store['임대료'].isna()].index
for idx, value in zip(missing_rent_indices, predicted_rent):
    train.at[idx, '임대료'] = value

missing_deposit_indices = train_non_store[train_non_store['임대보증금'].isna()].index
for idx, value in zip(missing_deposit_indices, predicted_deposit):
    train.at[idx, '임대보증금'] = value

# 결측치 채워졌는지 확인
missing_values = train[train['공급유형'] != '상가'][['임대료', '임대보증금']].isnull().sum()

missing_values

임대료      0
임대보증금    0
dtype: int64

In [101]:
# test 데이터 공급유형이 상가가 아닌 것만 남도록 필터링
test_non_store = test[test['공급유형'] != '상가']

# 지역명 추출
regions = test_non_store['지역'].unique()

# 예측값 저장을 위해 빈 리스트 생성
predicted_rent = [] #임대료 예측값 저장 리스트
predicted_deposit = [] #임대보증금 예측값 저장 리스트

# 지역을 기준으로 반복문 돌림
for region in regions:
    region_data = test_non_store[test_non_store['지역'] == region].copy()

    # 임대료에 대한 회귀 모델 학습
    X = region_data[~region_data['임대료'].isna()][['전용면적']]
    y = region_data[~region_data['임대료'].isna()]['임대료']
    if len(X) > 0:
        model_rent = LinearRegression()
        model_rent.fit(X, y)
        # 결측치 예측
        X_missing_rent = region_data[region_data['임대료'].isna()][['전용면적']]
        # 데이터가 존재하면 모델을 사용해 임대료를 예측하고 리스트에 추가하도록 함
        if len(X_missing_rent) > 0:
            pred_rent = model_rent.predict(X_missing_rent)
            predicted_rent.extend(pred_rent)

    # 임대보증금에 대해서도 동일하게 회귀 모델 학습
    X = region_data[~region_data['임대보증금'].isna()][['전용면적']]
    y = region_data[~region_data['임대보증금'].isna()]['임대보증금']
    if len(X) > 0:
        model_deposit = LinearRegression()
        model_deposit.fit(X, y)
        # 결측치 예측
        X_missing_deposit = region_data[region_data['임대보증금'].isna()][['전용면적']]
        # 데이터가 존재하면 모델을 사용해 임대보증금을 예측하고 리스트에 추가하도록 함
        if len(X_missing_deposit) > 0:
            pred_deposit = model_deposit.predict(X_missing_deposit)
            predicted_deposit.extend(pred_deposit)

# test_non_store 결측치를 예측한 임대료와 임대보증금 값으로 채움
missing_rent_indices = test_non_store[test_non_store['임대료'].isna()].index
for idx, value in zip(missing_rent_indices, predicted_rent):
    test.at[idx, '임대료'] = value

missing_deposit_indices = test_non_store[test_non_store['임대보증금'].isna()].index
for idx, value in zip(missing_deposit_indices, predicted_deposit):
    test.at[idx, '임대보증금'] = value

# 결측치 채워졌는지 확인
missing_values = test[test['공급유형'] != '상가'][['임대료', '임대보증금']].isnull().sum()

missing_values

임대료      0
임대보증금    0
dtype: int64

> train에는 있는데 test에는 없는 값들이 존재함
---
- 장기전세, 공공임대(5년), 공공분양
- 자격유형 : B, F, O
- 단지코드

In [102]:
train['공급유형'].value_counts()

국민임대         1758
임대상가          562
행복주택          213
공공임대(10년)     205
영구임대          152
공공임대(50년)      31
공공임대(분납)       12
장기전세            9
공공분양            7
공공임대(5년)        3
Name: 공급유형, dtype: int64

In [103]:
test['공급유형'].value_counts()

국민임대         622
임대상가         177
행복주택         124
영구임대          45
공공임대(10년)     35
공공임대(50년)     13
공공임대(분납)       6
Name: 공급유형, dtype: int64

 - 장기전세, 공공임대(5년), 공공분양이 train에서만 존재함
 -- '공공'으로 시작하는 거끼리 묶고, 장기전세는 장기적으로 주택을 임대, 국민 임대 역시 장기 임대한다는 점에서 둘을 묶기로 결정

In [104]:

train.loc[train.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)','공공임대(50년)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
test.loc[test.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)','공공임대(50년)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'

train.loc[train.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
test.loc[test.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'


In [105]:

print(train['공급유형'].value_counts())
print('-----'*10)
print(test['공급유형'].value_counts())
#column값 차이 없는 것 확인


국민임대/장기전세             1767
임대상가                   562
공공임대(5년/10년/분납/분양)     258
행복주택                   213
영구임대                   152
Name: 공급유형, dtype: int64
--------------------------------------------------
국민임대/장기전세             622
임대상가                  177
행복주택                  124
공공임대(5년/10년/분납/분양)     54
영구임대                   45
Name: 공급유형, dtype: int64


In [106]:
train['자격유형'].value_counts()
# B- 21
# F -3
# O -1
# 데이터가 적어서 바로 앞의 자격 유형과 묶어도 무방할 것 같음

A    1801
D     569
H     155
J     114
C      95
I      49
E      37
K      33
L      33
N      30
B      21
G       9
F       3
M       2
O       1
Name: 자격유형, dtype: int64

In [107]:
test['자격유형'].value_counts()

A    572
D    180
H     92
J     84
C     34
K     16
L     12
E     10
N     10
I      7
M      2
G      1
Name: 자격유형, dtype: int64

In [108]:
# B,F,O 각 자격유형의 바로 앞 자격유형과 통합함
train['자격유형'] = train['자격유형'].apply(lambda x: 'A-B' if x in ['A', 'B'] else x)
test['자격유형'] = test['자격유형'].apply(lambda x: 'A-B' if x in ['A', 'B'] else x)

train['자격유형'] = train['자격유형'].apply(lambda x: 'E-F' if x in ['E', 'F'] else x)
test['자격유형'] = test['자격유형'].apply(lambda x: 'E-F' if x in ['E', 'F'] else x)

train['자격유형'] = train['자격유형'].apply(lambda x: 'N-O' if x in ['N', 'O'] else x)
test['자격유형'] = test['자격유형'].apply(lambda x: 'N-O' if x in ['N', 'O'] else x)

train['자격유형'].value_counts()

A-B    1822
D       569
H       155
J       114
C        95
I        49
E-F      40
K        33
L        33
N-O      31
G         9
M         2
Name: 자격유형, dtype: int64

In [109]:
train.shape, train.drop_duplicates().shape #중복존재

((2952, 15), (2651, 15))

In [110]:
test.shape, test.drop_duplicates().shape #중복존재

((1022, 14), (952, 14))

In [111]:
train = train.drop_duplicates()
test=test.drop_duplicates()

> data type이 object인 것을 모두 인코딩(원-핫 인코딩 사용)

In [112]:
# '단지코드'를 제외하고 object인 열을 모두 원-핫 인코딩 진행

# 인코딩할 object 열을 식별
object_columns = train.select_dtypes(include=['object']).columns.tolist()
object_columns.remove('단지코드')  # '단지코드' 열은 제외하여 인코딩하지 않음

# train, test에 대해 원-핫 인코딩 진행
train_encoded = pd.get_dummies(train, columns=object_columns)
test_encoded = pd.get_dummies(test, columns=object_columns)

# 인코딩이 진행된 train 데이터의 열 확인
train_encoded.head()
train_encoded.columns

Index(['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '임대보증금', '임대료', '지하철수',
       '버스수', '단지내주차면수', '등록차량수', '임대건물구분_상가', '임대건물구분_아파트', '지역_강원도',
       '지역_경상도', '지역_수도권(서울/경기)', '지역_전라도', '지역_제주특별자치도', '지역_충청도',
       '공급유형_공공임대(5년/10년/분납/분양)', '공급유형_국민임대/장기전세', '공급유형_영구임대', '공급유형_임대상가',
       '공급유형_행복주택', '자격유형_A-B', '자격유형_C', '자격유형_D', '자격유형_E-F', '자격유형_G',
       '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K', '자격유형_L', '자격유형_M', '자격유형_N-O'],
      dtype='object')

In [113]:
# '임대건물구분'이 '상가'인 경우 '임대료', '임대보증금' 값을 drop
# '임대건물구분'이 '상가'가 아닌 경우는 drop없이 그대로 진행

train_store_data = train_encoded[train_encoded['임대건물구분_상가'] == 1].drop(columns=['임대료', '임대보증금'])
train_non_store_data = train_encoded[train_encoded['임대건물구분_상가'] != 1]

test_store_data = test_encoded[test_encoded['임대건물구분_상가'] == 1].drop(columns=['임대료', '임대보증금'])
test_non_store_data = test_encoded[test_encoded['임대건물구분_상가'] != 1]

### `임대건물구분`이 상가인 것을 학습데이터로 나누고, 정규화를 두가지 방법(minMax Scaler, Stndardization Scaler)로 진행한 후,
### Ridge, Lasso, ElasticNet 회귀 모델을 사용하여 MAE값이 가장 낮은것을 출력

In [114]:
#'임대건물구분'이 '상가'와 '상가가 아닌 것'으로 데이터를 분리
features_store = train_store_data.columns.drop(['등록차량수', '단지코드']).tolist()
features_nonstore = train_non_store_data.columns.drop(['등록차량수', '단지코드']).tolist()
target = '등록차량수'

X_store =train_store_data[features_store]
y_store=train_store_data[target]

X_non_store =train_non_store_data[features_nonstore]
y_non_store=train_non_store_data[target]

# 훈련 데이터와 검증 데이터로 분리
X_train_store, X_val_store, y_train_store, y_val_store = train_test_split(X_store, y_store, test_size=0.2, random_state=42)
X_train_non_store, X_val_non_store, y_train_non_store, y_val_non_store = train_test_split(X_non_store, y_non_store, test_size=0.2, random_state=42)

# Initialize scalers again
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Apply MinMax Scaling

X_train_store_minmax = minmax_scaler.fit_transform(X_train_store)
X_val_store_minmax = minmax_scaler.transform(X_val_store)

X_train_non_store_minmax = minmax_scaler.fit_transform(X_train_non_store)
X_val_non_store_minmax = minmax_scaler.transform(X_val_non_store)

# Apply Standard Scaling
X_train_store_std = standard_scaler.fit_transform(X_train_store)
X_val_store_std = standard_scaler.transform(X_val_store)

X_train_non_store_std = standard_scaler.fit_transform(X_train_non_store)
X_val_non_store_std = standard_scaler.transform(X_val_non_store)

In [115]:
import numpy as np
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# 모델 및 파라미터 그리드 설정
models = {
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet()
}

params = {
    'Lasso': {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
        'max_iter': [1000,5000,10000],
        'tol': [0.00001,0.0001, 0.001]
    },
    'Ridge': {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
        'max_iter': [1000,5000,10000],
        'tol': [0.00001,0.0001, 0.001]
    },
    'ElasticNet': {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
        'l1_ratio': [.1, .5, .7, .9, .95, .99, 1],
        'max_iter': [1000,5000,10000],
        'tol': [0.00001,0.0001, 0.001]
    }
}

# 스케일링 방법
scaling_methods = {
    'MinMax Scaling': (X_train_store_minmax, X_val_store_minmax),
    'Standard Scaling': (X_train_store_std, X_val_store_std)
}

# 그리드 서치 및 MAE 계산
best_mae = float('inf')
best_model = None
best_params = None
best_scaling = None

for name, model in models.items():
    for method, (X_train_store_scaled, X_val_store_scaled) in scaling_methods.items():
        grid = GridSearchCV(model, params[name], cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')
        grid.fit(X_train_store_scaled, y_train_store)
        y_pred_store = grid.predict(X_val_store_scaled)
        mae = mean_absolute_error(y_val_store, y_pred_store)

        if mae < best_mae:
            best_mae_store = mae
            best_model_store = name
            best_params_store = grid.best_params_
            best_scaling_store = method

print(f"Best Model: {best_model_store}")
print(f"Best Scaling Method: {best_scaling_store}")
print(f"Best Parameters: {best_params_store}")
print(f"Best MAE: {best_mae_store}")


Best Model: ElasticNet
Best Scaling Method: Standard Scaling
Best Parameters: {'alpha': 1, 'l1_ratio': 1, 'max_iter': 1000, 'tol': 1e-05}
Best MAE: 45.88403485952757


In [116]:
import numpy as np
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# 데이터 및 스케일러가 주어진 것으로 가정
# X_train, X_val, y_train, y_val = ...
# minmax_scaler = ...
# standard_scaler = ...

# 모델 및 파라미터 그리드 설정
models = {
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet()
}

params = {
    'Lasso': {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
        'max_iter': [100,500,1000,5000,10000],
        'tol': [0.00001,0.0001, 0.001]
    },
    'Ridge': {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
        'max_iter': [100, 500,1000,5000,10000],
        'tol': [0.00001,0.0001, 0.001]
    },
    'ElasticNet': {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
        'l1_ratio': [.1, .5, .7, .9, .95, .99, 1],
        'max_iter': [500,1000,5000,10000],
        'tol': [0.00001,0.0001, 0.001]
    }
}

# 스케일링 방법
scaling_methods = {
    'MinMax Scaling': (X_train_non_store_minmax, X_val_non_store_minmax),
    'Standard Scaling': (X_train_non_store_std, X_val_non_store_std)
}

# 그리드 서치 및 MAE 계산
best_mae = float('inf')
best_model = None
best_params = None
best_scaling = None

for name, model in models.items():
    for method, (X_train_non_store_scaled, X_val_non_store_scaled) in scaling_methods.items():
        grid = GridSearchCV(model, params[name], cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')
        grid.fit(X_train_non_store_scaled, y_train_non_store)
        y_pred_non_store = grid.predict(X_val_non_store_scaled)
        mae = mean_absolute_error(y_val_non_store, y_pred_non_store)

        if mae < best_mae:
            best_mae_non_store = mae
            best_model_non_store = name
            best_params_non_store  = grid.best_params_
            best_scaling_non_store  = method

print(f"Best Model: {best_model_non_store}")
print(f"Best Scaling Method: {best_scaling_non_store}")
print(f"Best Parameters: {best_params_non_store}")
print(f"Best MAE: {best_mae_non_store}")

best_mae_store, best_mae_non_store

Best Model: ElasticNet
Best Scaling Method: Standard Scaling
Best Parameters: {'alpha': 1, 'l1_ratio': 0.99, 'max_iter': 500, 'tol': 0.0001}
Best MAE: 133.91135885663851


(45.88403485952757, 133.91135885663851)

In [117]:
X_test_store = test_store_data[features_store]
X_test_non_store = test_non_store_data[features_nonstore]

In [118]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 인스턴스 생성
standard_scaler = StandardScaler()

# 학습 데이터에 대해 스케일러를 학습(fit)시킴
standard_scaler.fit(X_train_store)

StandardScaler()

In [119]:
# 선택한 모델로 스케일링된 테스트 데이터에 대한 예측 수행 (상가)
X_test_store_scaled = None
standard_scaler = StandardScaler()
standard_scaler.fit(X_test_store)
if best_scaling_store == 'MinMax Scaling':
    X_test_store_scaled = minmax_scaler.transform(X_test_store)
elif best_scaling_store == 'Standard Scaling':
    X_test_store_scaled = standard_scaler.transform(X_test_store)

y_pred_test_store = None

if best_model_store == 'Lasso':
    model_store = Lasso(**best_params_store)
elif best_model_store == 'Ridge':
    model_store = Ridge(**best_params_store)
elif best_model_store == 'ElasticNet':
    model_store = ElasticNet(**best_params_store)

model_store.fit(X_train_store_scaled, y_train_store)
y_pred_test_store = model_store.predict(X_test_store_scaled)


array([ 84.96105752,  84.96105752,  84.96105752,  84.96105752,
        84.96105752,  84.96105752,  84.96105752,  84.96105752,
        84.96105752,  84.96105752,  84.96105752,  84.96105752,
        84.96105752, 284.7022268 , 284.7022268 , 284.7022268 ,
       284.7022268 , 284.7022268 , 284.7022268 , 284.7022268 ,
       284.7022268 , 284.7022268 , 284.7022268 , 284.7022268 ,
       284.7022268 , 284.7022268 , 284.7022268 , 284.7022268 ,
       284.7022268 , 284.7022268 , 284.7022268 , 284.7022268 ,
       284.7022268 , 284.7022268 , 284.7022268 , 284.7022268 ,
       284.7022268 , 284.7022268 , 284.7022268 , 284.7022268 ,
        42.74350225,  42.74350225,  42.74350225,  42.74350225,
        42.74350225,  42.74350225,  42.74350225,  42.74350225,
        42.74350225,  42.74350225,  42.74350225,  42.74350225,
        42.74350225,  42.74350225,  42.74350225, 238.50587513,
       238.50587513, 238.50587513, 238.50587513, 238.50587513,
       238.50587513, 238.50587513, 238.50587513, 238.50

In [123]:
# 선택한 모델로 스케일링된 테스트 데이터에 대한 예측 수행 (상가가 아닌 것)
standard_scaler = StandardScaler()

# 학습 데이터에 대해 스케일러를 학습(fit)시킴
standard_scaler.fit(X_train_non_store)
X_test_non_store_scaled = None

if best_scaling_non_store == 'MinMax Scaling':
    X_test_non_store_scaled = minmax_scaler.transform(X_test_non_store)
elif best_scaling_non_store == 'Standard Scaling':
    X_test_non_store_scaled = standard_scaler.transform(X_test_non_store)

y_pred_test_non_store = None

if best_model_non_store == 'Lasso':
    model_non_store = Lasso(**best_params_non_store)
elif best_model_non_store == 'Ridge':
    model_non_store = Ridge(**best_params_non_store)
elif best_model_non_store == 'ElasticNet':
    model_non_store = ElasticNet(**best_params_non_store)

model_non_store.fit(X_train_non_store_scaled, y_train_non_store)
y_pred_test_non_store = model_non_store.predict(X_test_non_store_scaled)

y_pred_test_store[y_pred_test_store < 0] = 0
y_pred_test_non_store[y_pred_test_non_store < 0] = 0

# 로그 변환
y_pred_test_store = np.log1p(y_pred_test_store)
y_pred_test_non_store = np.log1p(y_pred_test_non_store)

# 역 로그 변환
y_pred_test_store = np.expm1(y_pred_test_store)
y_pred_test_non_store = np.expm1(y_pred_test_non_store)

# 예측 결과를 원래 데이터 프레임에 추가
test_store_data['예측등록차량수'] = y_pred_test_store
test_non_store_data['예측등록차량수'] = y_pred_test_non_store


# 예측 결과를 원래 데이터 프레임에 추가
test_store_data['예측등록차량수'] = y_pred_test_store
test_non_store_data['예측등록차량수'] = y_pred_test_non_store

# 예측 결과 병합
test_encoded_combined = pd.concat([test_store_data, test_non_store_data])

# 각 단지코드의 예측된 등록차량수의 평균을 계산
submission = test_encoded_combined.groupby('단지코드')['예측등록차량수'].mean().reset_index()
submission.columns=['code','num']

# sample submission 양식에 맞게 결과 조정
final_submission = sample_submission.merge(submission, on='code', how='left')
final_submission = final_submission[['code', 'num_y']]
final_submission.columns = ['code', 'num']

final_submission.head()
# 예측 결과를 csv 파일로 저장
final_submission.to_csv('submission3.csv', index=False)